https://colab.research.google.com/drive/129VHNnTovtentiO3SZHHUzUcALje0-w9

The images and captions data can be found here:
http://academictorrents.com/details/9dea07ba660a722ae1008c4c8afdd303b6f6e53b

In [1]:
import glob
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
from tqdm import tqdm
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, merge, Activation, Flatten
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
import nltk

Using TensorFlow backend.


In [0]:
token = '...'  # Path to captions file

In [0]:
captions = open(token, 'r').read().strip().split('\n')

In [0]:
d = {}
for i, row in enumerate(captions):
    row = row.split('\t')
    row[0] = row[0][:len(row[0])-2]
    if row[0] in d:
        d[row[0]].append(row[1])
    else:
        d[row[0]] = [row[1]]

In [0]:
total_images = list(d.keys())

In [0]:
from keras.applications.vgg16 import VGG16
vgg16=VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))

Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 6s 0us/step


In [0]:
import cv2
x = []
y = []
x2 = []
tot_images = ''
for i in range(len(d.keys())):
#for i in range(1000):
  if(i%100)==0:
    print(i)
  for j in range(len(d[total_images[i]])):
    img_path = '...'   # Path to images
    img = cv2.imread(img_path)
    try:
      img2 = cv2.resize(img, (224, 224))/255
      img3 = vgg16.predict(img2.reshape(1,224,224,3))
      #x.append(img)
      x2.append(img3)
      y.append(d[total_images[i]][j])
      tot_images = tot_images + ' '+total_images[i]
    except:
      print(1)

In [0]:
x2 = np.array(x2)
x2.shape

(40455, 1, 7, 7, 512)

In [0]:
x2 = x2.reshape(x2.shape[0],7,7,512)

In [0]:
img_path2 = tot_images.split()

In [0]:
import nltk
nltk.download('stopwords')
stop = nltk.corpus.stopwords.words('english')
import re

def preprocess(text):
 text=text.lower()
 text=re.sub('[^0-9a-zA-Z]+',' ',text)
 words = text.split()
 #words2=[w for w in words if (w not in stop)]
 words2 = words
 #words3=[ps.stem(w) for w in words]
 words4=' '.join(words2)
 return(words4)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
caps = []
for key, val in d.items():
  if(key in img_path2):
    for i in val:
      i = preprocess(i)
      caps.append('<start> ' + i + ' <end>')

In [0]:
len(caps)

40455

In [0]:
word = ['child','girl']

In [0]:
caps2 = []
x3 = []
img_path3 = []
for i in range(len(caps)):
  if (('girl') in caps[i]):
    caps2.append(caps[i])
    x3.append(x2[i])
    img_path3.append(img_path2[i])
  elif 'dog' in caps[i]:
    caps2.append(caps[i])
    x3.append(x2[i])
    img_path3.append(img_path2[i])
  else:
    continue

In [0]:
len(caps2)

13445

In [0]:
len(img_path3)

13445

In [0]:
x3 = np.array(x3)

In [0]:
words = [i.split() for i in caps2]
unique = []
for i in words:
    unique.extend(i)
	
unique = list(set(unique))
len(unique)

3935

In [0]:
word2idx = {val:(index+1) for index, val in enumerate(unique)}

idx2word = {(index+1):val for index, val in enumerate(unique)}

max_len = 0
for c in caps2:
    c = c.split()
    if len(c) > max_len:
        max_len = len(c)
max_len

35

In [0]:
caps2[0]

'<start> a girl going into a wooden building <end>'

In [0]:
vocab_size = len(unique)
vocab_size

3935

In [0]:
n = np.zeros(vocab_size+1)
n.shape

(3936,)

In [0]:
word2idx['<end>']

3735

In [0]:
len(caps2)

13445

In [0]:
y = []
y2 = []
for k in range(len(caps2)):
  t= [word2idx[i] for i in caps2[k].split()]
  y.append(len(t))
  while(len(t)<max_len):
    t.append(word2idx['<end>'])
  y2.append(t)

In [0]:
len(y2[0])

35

In [0]:
import random

In [0]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical,np_utils
from keras.layers import Dense, Input, GlobalMaxPooling1D,Dropout, Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Convolution2D, MaxPooling2D,Conv2D
from keras.models import Model
import random
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM , Bidirectional,Dropout
from keras import backend as K
from keras.layers.advanced_activations import LeakyReLU
from keras import regularizers
from keras.layers import concatenate

In [0]:
from keras.layers import Input
embedding_size = 300
inp = Input(shape=(7,7,512))
inp1 = Conv2D(512, (3,3), activation='relu')(inp)
inp11 = MaxPooling2D(pool_size=(2, 2))(inp1)
inp2 = Flatten()(inp11)
img_emb = Dense(embedding_size, activation='relu') (inp2)
img_emb2 = RepeatVector(max_len)(img_emb)

In [0]:
inp2 = Input(shape=(max_len,))
cap_emb = Embedding((vocab_size+1), embedding_size, input_length=max_len) (inp2)
cap_emb2 = LSTM(256, return_sequences=True)(cap_emb)
cap_emb3 = TimeDistributed(Dense(300)) (cap_emb2)

In [0]:
final1 = concatenate([img_emb2, cap_emb3])
final2 = Bidirectional(LSTM(256, return_sequences=False))(final1)
final3 = Dense(vocab_size+1)(final2)
final4 = Activation('softmax')(final3)

final_model = Model([inp, inp2], final4)
final_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 7, 7, 512)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 5, 5, 512)    2359808     input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 2, 2, 512)    0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 35)           0                                            
__________________________________________________________________________________________________
flatten_1 

In [0]:
from keras.optimizers import Adam
adam = Adam(lr = 0.0001)

In [0]:
final_model.compile(loss='categorical_crossentropy', optimizer = adam, metrics=['accuracy'])

In [0]:
for i in range(2500):
  x4 = []
  x4_sent = []
  y3 = []
  shortlist_y = random.sample(range(len(y)-100),32)
  for j in range(len(shortlist_y)):
    for k in range(y[shortlist_y[j]]-1):
      n = np.zeros(vocab_size+1)      
      x4.append(x3[shortlist_y[j]])
      pad_sent = pad_sequences([y2[shortlist_y[j]][:(k+1)]], maxlen=max_len, padding='post')
      x4_sent.append(pad_sent)
      n[y2[shortlist_y[j]][(k+1)]] = 1
      y3.append(n)
  x4 = np.array(x4)
  x4_sent =np.array(x4_sent)
  x4_sent = x4_sent.reshape(x4_sent.shape[0], x4_sent.shape[2])
  y3 = np.array(y3) 
  if(i%100==0):
    print(i)
    final_model.fit([x4/12, x4_sent], y3, batch_size = 32, epochs = 3, verbose = 1)
  else:
    final_model.fit([x4/12, x4_sent], y3, batch_size = 32, epochs = 3, verbose = 0)  
  

0
Instructions for updating:
Use tf.cast instead.
Epoch 1/3
361/361 [==============================] - 6s 17ms/step - loss: 8.2445 - acc: 0.0886
Epoch 2/3
361/361 [==============================] - 3s 8ms/step - loss: 7.9514 - acc: 0.1524
Epoch 3/3
361/361 [==============================] - 3s 8ms/step - loss: 7.0352 - acc: 0.1524
100
Epoch 1/3
343/343 [==============================] - 3s 8ms/step - loss: 4.5160 - acc: 0.2012
Epoch 2/3
343/343 [==============================] - 3s 8ms/step - loss: 4.3533 - acc: 0.2187
Epoch 3/3
343/343 [==============================] - 3s 8ms/step - loss: 4.1801 - acc: 0.2274
200
Epoch 1/3
350/350 [==============================] - 3s 8ms/step - loss: 4.0853 - acc: 0.2886
Epoch 2/3
350/350 [==============================] - 3s 8ms/step - loss: 3.9505 - acc: 0.2943
Epoch 3/3
350/350 [==============================] - 3s 8ms/step - loss: 3.7938 - acc: 0.3000
300
Epoch 1/3
407/407 [==============================] - 3s 8ms/step - loss: 4.1039 - acc: 0.26

In [0]:
p = np.zeros(max_len)
p[0] = word2idx['<start>']
l=-91

In [0]:
im_path = '...'  # Path to test image file
img1 = cv2.imread(im_path)
plt.imshow(img1)
plt.grid('off')

In [0]:
caps2[l]

'<start> the girl wearing the orange shirt and glasses is blowing a huge bubble <end>'

In [0]:
x33= np.array(x3)
for i in range(max_len-1):
  pred= final_model.predict([x33[l].reshape(1,7,7,512)/12, p.reshape(1,max_len)])
  pred2 = np.argmax(pred)
  print(idx2word[pred2])
  p[i+1] = pred2

a
little
girl
in
a
pink
shirt
runs
in
a
air
<end>
<end>
<end>
<end>
<end>
<end>
<end>
<end>
<end>
<end>
<end>
<end>
<end>
<end>
<end>
<end>
<end>
<end>
<end>
<end>
<end>
<end>
<end>


# Beam search

In [0]:
beamsize = 3
def get_top3(img, string_with_conf):
     tokens, confidence = string_with_conf
     p = np.zeros((1, max_len))
     p[0, :len(tokens)] = np.array(tokens)
     pred = final_model.predict([img.reshape(1,7,7,512)/12, p])
     best_pred = list(np.argsort(pred)[0][-beamsize:])
     best_confs = list(pred[0,best_pred])
     top_best = [(tokens + list([best_pred[i]]), confidence*best_confs[i]) for i in range(beamsize)]
     return top_best

In [0]:
start_token = word2idx['<start>']
best_strings = [([start_token], 1)]
for i in range(max_len):
     new_best_strings = []
     for string in best_strings:
         strings = get_top3(x33[l], string)
         new_best_strings.extend(strings) 
         best_strings = sorted(new_best_strings, key=lambda x: x[1], reverse=True)[:beamsize]

In [0]:
for i in range(3):
     string = best_strings[i][0]
     print('============')
     for j in string:
         print(idx2word[j])
         if(idx2word[j]=='<end>'):
             break